# 数据处理

## 数据处理 - 吸烟

smoking.xlsx

> col 0	            6位数字ID，前两位代表“县”  
> col 1和col 2合起来   表示地点  
> col 3	            烟税
> 
> 需要计算当地的人均烟税额，和该县的人均烟税额

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)   # 每行最大字符
pd.set_option('precision', 3)         # 显示数字精度

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms

plt.rcParams['figure.figsize'] = 18, 9
plt.rcParams['axes.unicode_minus'] = False     # 显示数字负号
plt.rcParams['font.sans-serif'] = ['SimHei']   # 显示中文字体
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['font.size'] = 12
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'
plt.style.use('seaborn-whitegrid')

import seaborn as sns


### 预览

In [2]:
df = pd.read_excel("../smoking.xlsx", sheet_name="Sheet1")

In [6]:
print('预览前几行')
df.head(10)

预览前几行


,団体 コード,Unnamed: 1,団体名,市町村たばこ税
0,11002,北海道,札幌市,672866
1,12025,北海道,函館市,79285
2,12033,北海道,小樽市,31130
3,12041,北海道,旭川市,98147
4,12050,北海道,室蘭市,28204
5,12068,北海道,釧路市,50656
6,12076,北海道,帯広市,55389
7,12084,北海道,北見市,37230
8,12092,北海道,夕張市,1836
9,12106,北海道,岩見沢市,23309


In [4]:
def explore_df(df):
  print('==== row, col', df.shape)
  print('==== describe ====')
  print(df.describe())
  print('==== info ====')
  print(df.info())
  
# explore_df(df)

In [5]:
explore_df(df)

==== row, col (1741, 4)
==== describe ====
          団体\nコード
count    1741.000
mean   211462.809
std    143348.780
min     11002.000
25%     82341.000
50%    204030.000
75%    332038.000
max    473821.000
==== info ====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1741 entries, 0 to 1740
Data columns (total 4 columns):
団体
コード        1741 non-null int64
Unnamed: 1    1741 non-null object
団体名           1741 non-null object
市町村たばこ税       1741 non-null object
dtypes: int64(1), object(3)
memory usage: 54.5+ KB
None
